<a href="https://colab.research.google.com/github/adeebafatima/Music-Composer/blob/main/Train_Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import json
import numpy as np
import pandas as pd

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/My Drive/Music Composer

/content/drive/My Drive/Music Composer


In [22]:
%ls

Code/  Data/


In [13]:
data_directory = "Data"
data_file = "Data/Input_Music_Tunes.txt"
char_index_json_file = "Data/char_to_index.json"
model_weights_location = 'Data/Model1/Model1_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [14]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #129665/16 = 8104 //blocks
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 8104, 64)  #it denotes number of batches. It runs everytime when new batch is created. We have a total of 126 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 86)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i + 1]
        yield X, Y

In [15]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(128, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [16]:
def train_model(data,epochs=1,save_freq=10):
    char_to_index = { ch: i for (i, ch) in enumerate(sorted(list(set(data)))) }
    print("Number of unique characters in our whole tunes train data = {}".format(len(char_to_index)))
    
    with open(char_index_json_file, mode = "w") as f:
        json.dump(char_to_index, f)
    
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #129665
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            model.save_weights((model_weights_location+"Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("Data/Model1/log1.csv", index = False)
    
    

In [17]:
file = open(data_file, mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    train_model(data)

Number of unique characters in our whole tunes train data = 86
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (16, 64, 512)             44032     
_________________________________________________________________
lstm_2 (LSTM)                (16, 64, 256)             787456    
_________________________________________________________________
dropout_2 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_3 (LSTM)                (16, 64, 128)             197120    
_________________________________________________________________
dropout_3 (Dropout)          (16, 64, 128)             0         
_________________________________________________________________
time_distributed_1 (TimeDist (16, 64, 86)              11094     
_________________________________________________________

In [21]:
log = pd.read_csv("Data/Model1/log1.csv")
log

,Epoch,Loss,Accuracy
0,1,2.710444,0.275391
1,2,1.943644,0.466797
2,3,1.651903,0.532227
3,4,1.557028,0.549805
4,5,1.509097,0.562500
...,...,...,...
95,96,0.461089,0.844727
96,97,0.473274,0.837891
97,98,0.474931,0.844727
98,99,0.485827,0.849609
